In [1]:
import pickle
import datetime
import pandas.io.data as web
import sys
import pandas as pd
sys.path.append('../prod/')
from Portfolio import Portfolio, Position

Here is what this file does:
(1) given an array of arrays of tuples (weight, ticker,dates) found in portfolios.txt, calculates historical returns of the fund by tracking the performance of the portfolio
(2) for the most recent one, compute all static metrics (those who only depend on daily data) displayed by the website:
    - Historical Performance
    - Markowitz
    - Monte Carlo
    - Scalars
    - Multi Factor Risk Model

Updates in portfolio happen when the manager changes his positions manually. only these are kept track of in the .txt file.
--> Should have a marker for when the second type of updates happen. display those updates on the historical performance chart (red dots)

Everytime the portfolio is updated (at least once a day), a new portfolio is added to portfolio.txt from update.py. then the methods in this file are triggered and update the csv of returns before plotting new charts.

To be data efficient, the returns data will solely be held by the csv file. no portfolio object will keep track of the returns of each of its positions.

In [2]:
#requirement when .csv file is empty: load one series in
df = web.DataReader(['GOOGL','GLD'],'yahoo',datetime.datetime(2010, 1, 1))['Close']
df = df.pct_change()[1:len(df)]
df.columns = ['GOOGL','GLD']
df.to_csv("returns_data.csv")

###Manager Updates

In [3]:
P1 = (['GOOGL','FB','GLD'],[0.5,0.25,0.25],[792,128,1200])
#where p is just like P1 : (tickers,initial weights, price of purchase)
def start_manager_update_process(p):
    latest_portfolio = manager_portfolio_update(p)
    save_new_portfolio(latest_portfolio)
    make_new_charts(latest_portfolio)

def manager_portfolio_update(p):
    tickers = p[0]
    weights = p[1]
    costs = p[2]
    df = manager_data_update(tickers)
    df = df.dropna(thresh=len(tickers), axis=0) #this line restrict historical calculations to data points shared by all equities
    all_dates = df.index
    assets = []
    for i in range(len(tickers)):
        symbol = tickers[i]
        w = weights[i]
        in_price = costs[i]
        returns = df[symbol]
        direction_pos = direction(w)
        assets.append(Position(symbol,returns,direction_pos,in_price,w))
    #start_date = datetime.date.today()
    start_date = datetime.datetime(2016,10,25)
    latest_portfolio = Portfolio(assets,start_date,all_dates)
    return latest_portfolio
    
#initially load csv with one series going back at least to 2010
def manager_data_update(tickers):
    df = pd.DataFrame.from_csv("returns_data.csv")
    tickers = set(tickers)
    new_tickers = list(tickers - set(df.columns))
    new_columns = web.DataReader(new_tickers,'yahoo',datetime.datetime(2010, 1, 1))['Close']
    new_columns = new_columns.pct_change()[1:len(new_columns)]
    df = df.join(new_columns,how='outer')
    df.to_csv("returns_data.csv")
    return df

def save_new_portfolio(latest_portfolio):
    with open('portfolio.txt', 'wb') as f:
        all_portfolios = pickle.load(f)
        all_portfolios.append(latest_portfolio.compile_portfolio())
        pickle.dump(all_portfolios, f)

###General

In [2]:
def direction(w):
    if w >= 0:
        return "long"
    else:
        return "short"

def make_new_charts(latest_portfolio):
    return 'done'

#use this to plot the historical performance of the fund, for whatever asset allocation
def pull_old_portfolios():
    with open('portfolio.txt','rb') as f:
        var = pickle.load(f)
    return var

### Daily Updates

In [5]:
def start_daily_update_process():
    latest_saved_portfolio = pull_latest_portfolio()
    df = daily_data_updates()
    latest_portfolio = daily_portfolio_update(latest_saved_portfolio,df)
    update_last_portfolio(latest_portfolio)

def daily_portfolio_update(latest_saved_portfolio,return_grid):
    df = return_grid
    latest_portfolio = latest_saved_portfolio.uncompile(df)
    return latest_portfolio
    
def pull_latest_portfolio():
    olds = pull_old_portfolios()
    return olds[len(olds)-1]

def daily_data_update():
    df = pd.DataFrame.from_csv("returns_data.csv")
    tickers = df.columns
    today_row = web.DataReader(tickers,'yahoo',datetime.date.today() - datetime.timedelta(days=1),datetime.date.today())['Close']
    today_row = today_row.pct_change()[1:len(today_row)]
    df = df.append(today_row)
    df.to_csv("returns_data.csv")
    return df

def update_last_portfolio(latest_portfolio):
    with open('portfolio.txt', 'wb') as f:
        all_portfolios = pickle.load(f)
        all_portfolios[len(all_portfolios)-1] = latest_portfolio.compile_portfolio()
        pickle.dump(all_portfolios, f)